# <font size="10">Custom entity recognition </font>
## Model environment setup

This notebook contains test code to train the implemented model in the generated training data outputted from
the [ner-train notebook located here](./ner-train-note.ipynb).

For simple loop model training (old), go [here](## Run training - using simple training loop from blank -- Old)

## Setup

In [1]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=5d1789a1a005138d3bf4ac53a6eb5d2c67fc22b21477dd818ac3cb87adde51ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-tl1oeale/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
import pandas as pd
import random
import warnings
import json
import ast
import datetime as dt
from pathlib import Path
import os
import glob

##SpaCy

import en_core_web_sm
import en_core_web_md

import spacy
from spacy import displacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.pipeline import Sentencizer
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB
from spacy.util import minibatch, compounding
from spacy.util import decaying


# For Colab

In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import training data

lets import the training data we generated:

In [4]:
train_path = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/'

csvs = glob.glob(train_path + "/*.csv")


print('Filepath is :',(csvs))

train_list = []

for filename in csvs:
    df = pd.read_csv(filename, index_col=None, header=0)
    train_list.append(df)


Filepath is : ['/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset0.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset1.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset2.csv']


In [5]:
DATA = pd.concat(train_list, axis=0, ignore_index=True)
DATA = DATA.drop_duplicates()
DATA[0:10]
print(len(DATA))

2101


In [6]:
#convert to list for model intake
TRAIN_DATA = DATA.values.tolist()

#for element in index 1 convert string (Entity position) to dictionary to be able to read by the model function
for position in TRAIN_DATA:
    position[1]=ast.literal_eval(position[1])
    
#Check our input list
print(TRAIN_DATA[0:2])

[['With increasing severe weather events and disasters \ntriggering greater numbers of costly power outages, there is a growing interest in generators for \nreliable backup power.', {'entities': [(161, 173, 'SUSTECH')]}], ['$3,153 \n\nNPC of Backup Power per Unit ($/kW)', {'entities': [(16, 28, 'SUSTECH')]}]]


## Run a test before training
### Test existing default spacy model 

In [7]:
nlp = en_core_web_md.load()
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f8725762ef0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f87256744c8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f8725674528>)]

In [8]:
doc = nlp('Here is a green roof on this house. A green roof is good.')
displacy.render(doc, style="ent")
# verified green roof does not match an entity in the NER

/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


'<div class="entities" style="line-height: 2.5; direction: ltr">Here is a green roof on this house. A green roof is good.</div>'

## TRAINING THE MODEL
## Train model setup

### Define compounding batch size

In [9]:
def get_batches(train_data, model_type):
    max_batch_sizes = {"tagger": 32, "parser": 16, "ner": 16, "textcat": 64}
    max_batch_size = max_batch_sizes[model_type]
    if len(train_data) < 1000:
        max_batch_size /= 2
    if len(train_data) < 500:
        max_batch_size /= 2
    batch_size = compounding(1, max_batch_size, 1.001)
    batches = minibatch(train_data, size=batch_size)
    return batches

In [10]:
#New training model loop to either accept existing model, if not model is not defined then create a blank nlp model using english vocab

def train_model(**model_params):

    model = model_params['model']
    iterations = model_params['iterations']
    train_data = model_params['train_data']
    dropout = model_params['dropout']
    
    
    random.seed(0)
    
    if model is not None:
        nlp = model.load() #load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise,get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    #ner.add_label(LABEL)  # add new entity label to entity recognizer
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            #print(ent[2])

    # Adding extraneous labels shouldn't mess anything up
    ner.add_label("VEGETABLE")
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes=compounding(4.0, 32.0, 1.001)
        # batch up the examples using spaCy's minibatch
                  
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.resume_training()

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        for itn in range(iterations):
            random.shuffle(train_data)
            #batches = get_batches(TRAIN_DATA, 'ner') resulted in poor loss
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, 
                           annotations,
                           sgd=optimizer, 
                           drop = 0.2, 
                           losses=losses)
            print(f"Losses at iteration {itn} - {dt.datetime.now()} {losses}")
    
    print('Model training completed')
    return nlp

In [11]:
model_params = {
    'model': None,
    'iterations': 30,
    'train_data': TRAIN_DATA,
    'dropout': decaying(0.6, 0.2, 1e-4)
}

In [12]:
nlp = train_model(**model_params)

Loaded model '<module 'en_core_web_sm' from '/usr/local/lib/python3.6/dist-packages/en_core_web_sm/__init__.py'>'
Losses at iteration 0 - 2020-10-28 17:42:51.093401 {'ner': 11511.718416785261}
Losses at iteration 1 - 2020-10-28 17:43:11.637572 {'ner': 9829.748458227154}
Losses at iteration 2 - 2020-10-28 17:43:32.120636 {'ner': 9761.455526038786}
Losses at iteration 3 - 2020-10-28 17:43:53.638535 {'ner': 9533.85092523863}
Losses at iteration 4 - 2020-10-28 17:44:16.649330 {'ner': 10355.43978221532}
Losses at iteration 5 - 2020-10-28 17:44:40.162388 {'ner': 10034.753038801666}
Losses at iteration 6 - 2020-10-28 17:45:03.998305 {'ner': 10206.816393882504}
Losses at iteration 7 - 2020-10-28 17:45:27.940386 {'ner': 10171.015126861636}
Losses at iteration 8 - 2020-10-28 17:45:51.995034 {'ner': 10370.28548294943}
Losses at iteration 9 - 2020-10-28 17:46:15.926139 {'ner': 10160.320187104153}
Losses at iteration 10 - 2020-10-28 17:46:39.855232 {'ner': 10058.46376944392}
Losses at iteration 11 

## Test the trained model

In [13]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f87121f8748>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f8712144588>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f87121445e8>)]

In [14]:
doc = nlp('Here is a green roof on this house. A green roof is good. water piping, I have alot of battery packs')


In [15]:
displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr">Here is a \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    green roof\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">SUSTECH</span>\n</mark>\n on this house. A \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    green roof\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">SUSTECH</span>\n</mark>\n is good. water piping, I have alot of battery packs</div>'

## Save model for testing

In [16]:
output_dir = r'./model'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to model


## Loading and testing the saved model

In [17]:
output_dir = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model'
x = ['i am a green roof']
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

for text in x:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model


FileNotFoundError: ignored